In [ ]:
#Import libraries

import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix)

#Import data
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')


#Remove unnecessary columns (customerID is an identifier)
df.drop(['customerID'], axis=1, inplace=True)

#Group features

#Online Security, Online Backup, Device Protection, Tech Support
def combined_securitysupport(row):
    if row['InternetService'] == 'No':
        return 'No internet service'
    
    count_yes = sum([
        row['OnlineSecurity'] == 'Yes',
        row['OnlineBackup'] == 'Yes',
        row['DeviceProtection'] == 'Yes',
        row['TechSupport'] == 'Yes'
    ])
    
    if count_yes == 0:
        return 'None'
    elif count_yes == 4:
        return 'All'
    else:
        return 'Some'

#Streaming TV, Streaming Movies
def combined_entertainment(row):
    if row['InternetService'] == 'No':
        return 'No internet service'
    
    count_yes = sum([
        row['StreamingTV'] == 'Yes',
        row['StreamingMovies'] == 'Yes'
    ])
    
    if count_yes == 0:
        return 'None'
    elif count_yes == 2:
        return 'Both'
    else:
        return 'One'

#Multiple Lines, Phone Service, or No phone service
def combined_lineservice(row):
    if row['PhoneService'] == 'No':
        return 'No phone service'
    elif row['MultipleLines'] == 'Yes':
        return 'Multiple lines'
    else:
        return 'Single line'
    
#One-hot encode certain features

#Create new combined features
df['PhoneLineStatus'] = df.apply(combined_lineservice, axis=1)

df['SecuritySupportLevel'] = df.apply(combined_securitysupport, axis=1)

df['EntertainmentLevel'] = df.apply(combined_entertainment, axis=1)

#Categories for one-hot encoding
one_hot_columns = ['gender','Partner', 'Dependents', 'PaperlessBilling', 'PhoneService']

encoder = OneHotEncoder(sparse_output=False)

one_hot_encoded = encoder.fit_transform(df[one_hot_columns])

one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(one_hot_columns))

df_encoded = pd.concat([df, one_hot_df], axis=1)

one_hot_columns.append('MultipleLines')

df_encoded = df_encoded.drop(one_hot_columns, axis=1)

df_encoded = pd.get_dummies(df, columns=one_hot_columns, drop_first=True)

df = df_encoded.copy()


#Manual Ordinal Encoding
df['SecuritySupportLevel'] = df['SecuritySupportLevel'].map({
    'No internet service': 0,
    'None': 1, 
    'Some': 2,
    'All': 3
})

df['EntertainmentLevel'] = df['EntertainmentLevel'].map({
    'No internet service': 0,
    'None': 1,
    'One': 2,
    'Both': 3
})

df['PhoneLineStatus'] = df['PhoneLineStatus'].map({
    'No phone service': 0,
    'Single line': 1,
    'Multiple lines': 2
})

df['Contract'] = df['Contract'].map({
    'Month-to-month': 0, 
    'One year': 1, 
    'Two year': 2})

df['InternetService'] = df['InternetService'].map({
    'DSL': 1, 
    'Fiber optic': 2,    'No': 0})

#TODO: Independent and Dependent variables
#TODO: Train-test split
#TODO: Model training on training set
#TODO: Model evaluation on test set
#TODO: Visualizations
